In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [3]:
def get_edges(line):
    edge = line.strip().split(',')
    n1 = edge[0]
    n2 = edge[1]
    if n1 < n2:
         return (n1,n2)
    elif n1 > n2:
         return (n2,n1)
    else:
        pass #n1 == n2

def lista_aristas_dep(tupla):
    aristas = []
    for i in range(len(tupla[1])):
        aristas.append(((tupla[0][0],tupla[1][i][0],tupla[0][1]),('exists')))
        for j in range(i+1,len(tupla[1])):
            if tupla[1][i][0]< tupla[1][j][0]:
                aristas.append(((tupla[1][i][0],tupla[1][j][0],tupla[0][1]),('pending',tupla[0][0])))
            else:
                aristas.append(((tupla[1][j][0],tupla[1][i][0],tupla[0][1]),('pending',tupla[0][0])))
    return aristas

def filtro(tupla):
    return (len(tupla[1]) >=2 and 'exists' in tupla[1])

def triciclos(h):
    l = []
    for i in h[1]:
        if i != 'exists':
            l.append(((i[1],h[0][0],h[0][1]),h[0][2]))
    return l

def principal(sc,files):
    rdd = sc.parallelize([])
    for file_name in files:    
        file_rdd = sc.textFile(file_name).\
            map(get_edges).\
            filter(lambda x: x is not None).\
            distinct().map(lambda x: ((x[0],file_name),(x[1],file_name)))
        rdd = rdd.union(file_rdd).distinct()
    adj = rdd.groupByKey().mapValues(list).flatMap(lista_aristas_dep).\
        groupByKey().mapValues(list).filter(filtro).flatMap(triciclos)
    return adj.collect()

In [4]:
principal(sc,["g1.txt","g4.txt"])

[(('C', 'D', 'E'), 'g4.txt'),
 (('A', 'B', 'C'), 'g4.txt'),
 (('A', 'C', 'E'), 'g4.txt'),
 (('A', 'E', 'F'), 'g4.txt'),
 (('A', 'B', 'D'), 'g1.txt'),
 (('A', 'C', 'D'), 'g1.txt'),
 (('B', 'C', 'D'), 'g1.txt'),
 (('A', 'B', 'C'), 'g1.txt')]